<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 5)
plt.rcParams.update({'errorbar.capsize': 3})
plt.rcParams['xtick.labelsize']= 13
plt.rcParams['ytick.labelsize']= 13
plt.rcParams.update({'axes.labelsize': 16})

In [2]:
from astropy import time

In [3]:
from astroquery.ipac.nexsci.nasa_exoplanet_archive import NasaExoplanetArchive

In [4]:
transit_date=time.Time(['2022-03-06T00:00:00',], format='isot', scale='utc').jd

In [5]:
transit_date

array([2459644.5])

In [6]:
res=NasaExoplanetArchive.query_object("WASP-31 b",table='ps',regularize=False)

In [7]:
res['pl_orbper'], res['pl_tranmid']

(<Quantity [3.4      ,       nan, 3.4059095, 3.4058841, 3.4059096,
            3.405886 , 3.405909 ] day>,
 <Quantity [          nan,           nan, 2458554.30005, 2456183.80208,
            2455192.6887 , 2455873.86662,           nan] day>)

In [8]:
res.columns

<TableColumns names=('pl_name','pl_letter','hostname','hd_name','hip_name','tic_id','gaia_id','default_flag','pl_refname','sy_refname','disc_pubdate','disc_year','discoverymethod','disc_locale','disc_facility','disc_instrument','disc_telescope','disc_refname','ra','rastr','dec','decstr','glon','glat','elon','elat','pl_orbper','pl_orbpererr1','pl_orbpererr2','pl_orbperlim','pl_orbperstr','pl_orblpererr1','pl_orblper','pl_orblpererr2','pl_orblperlim','pl_orblperstr','pl_orbsmax','pl_orbsmaxerr1','pl_orbsmaxerr2','pl_orbsmaxlim','pl_orbsmaxstr','pl_orbincl','pl_orbinclerr1','pl_orbinclerr2','pl_orbincllim','pl_orbinclstr','pl_orbtper','pl_orbtpererr1','pl_orbtpererr2','pl_orbtperlim','pl_orbtperstr','pl_orbeccen','pl_orbeccenerr1','pl_orbeccenerr2','pl_orbeccenlim','pl_orbeccenstr','pl_eqt','pl_eqterr1','pl_eqterr2','pl_eqtlim','pl_eqtstr','pl_occdep','pl_occdeperr1','pl_occdeperr2','pl_occdeplim','pl_occdepstr','pl_insol','pl_insolerr1','pl_insolerr2','pl_insollim','pl_insolstr','pl_dens

In [9]:
eph=np.array((res['pl_tranmid'].value, res['pl_orbper'].value)).transpose()
eph


array([[           nan, 3.40000000e+00],
       [           nan,            nan],
       [2.45855430e+06, 3.40590950e+00],
       [2.45618380e+06, 3.40588410e+00],
       [2.45519269e+06, 3.40590960e+00],
       [2.45587387e+06, 3.40588600e+00],
       [           nan, 3.40590900e+00]])

In [10]:
def get_midpoints(ephemeris,date):
    '''ephermeris in the form (t0, period)
    t0,date in JD/MJD (but both have to be the same)'''
    tmids=[]
    for item in ephemeris:
        per=item[1]
        t0=item[0]
        
        delta_date=date-t0
        num_of_orbs=delta_date/per
        tmids.append(t0+np.round(num_of_orbs,0)*per)
    return tmids

In [11]:
tmids=get_midpoints(eph,transit_date)

In [12]:
tmids-np.nanmean(tmids)

array([[       nan],
       [       nan],
       [-0.0005062],
       [-0.0112706],
       [ 0.020951 ],
       [-0.0091742],
       [       nan]])

In [13]:
res['pl_refname']

<a refstr=SING_ET_AL__2016 href=https://ui.adsabs.harvard.edu/abs/2016Natur.529...59S/abstract target=ref>Sing et al. 2016</a>
<a refstr=BARSTOW_ET_AL__2017 href=https://ui.adsabs.harvard.edu/abs/2017ApJ...834...50B/abstract target=ref>Barstow et al. 2017</a>
<a refstr=PATEL__AMP__ESPINOZA_2022 href=https://ui.adsabs.harvard.edu/abs/2022AJ....163..228P/abstract target=ref>Patel &amp; Espinoza 2022</a>
<a refstr=KOKORI_ET_AL__2021 href=https://ui.adsabs.harvard.edu/abs/2021arXiv211013863K/abstract target=ref>Kokori et al. 2021</a>
<a refstr=ANDERSON_ET_AL__2011 href=https://ui.adsabs.harvard.edu/abs/2011A&A...531A..60A/abstract target=ref> Anderson et al. 2011</a>
<a refstr=BONOMO_ET_AL__2017 href=https://ui.adsabs.harvard.edu/abs/2017A&A...602A.107B/abstract target=ref>Bonomo et al. 2017</a>
<a refstr=BROWN_ET_AL__2012 href=https://ui.adsabs.harvard.edu/abs/2012MNRAS.423.1503B/abstract target=ref>Brown et al. 2012</a>


In [14]:
(tmids-np.nanmean(tmids))*24*60 # in minutes

array([[         nan],
       [         nan],
       [ -0.72892718],
       [-16.22966386],
       [ 30.16944006],
       [-13.21084768],
       [         nan]])

In [15]:
def find_issues(planet,date,threshold=10):
    '''date in YYYY-MM-DD
    or YYYY-MM-DDTHH:MM
    threshold in minutes'''
    transit_date=time.Time([date], format='isot', scale='utc').jd
    res=NasaExoplanetArchive.query_object(planet,table='ps',regularize=False)
    eph=np.array((res['pl_tranmid'].value, res['pl_orbper'].value)).transpose()
    tmids=get_midpoints(eph,transit_date)
    difs=(tmids-np.nanmean(tmids))*24*60 # in minutes
    if (np.max(difs)-np.min(difs))>threshold:
        print('issue with '+planet)
        print('offsets in minutes: ',difs)
        print('sources: ',res['pl_refname'])
        print(' ')

In [16]:
transit_date=time.Time(['2022-03-06T00:00'], format='isot', scale='utc').jd
transit_date

array([2459644.5])

In [17]:
for item in np.array(res['pl_tranmid'].value):
    print(item)

nan
nan
2458554.30005
2456183.80208
2455192.6887
2455873.86662
nan


In [18]:
res['pl_tranmid'].value[0]==np.nan

False

In [19]:
import math

In [20]:
res['pl_tranmid'].value[0]

nan

In [21]:
type(res['pl_tranmid'].value[0])

numpy.float64

In [22]:
res['pl_tranmid'].value[0]

AttributeError: 'numpy.float64' object has no attribute 'isnan'

In [23]:
np.isnan(res['pl_tranmid'].value[0])

True